In [1]:
#Importing the libraries
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import keras
import shutil, glob
from zipfile import ZipFile
import imgaug.augmenters as iaa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Unzipping the Combined Datasets
combined1_zip_path = "/content/drive/MyDrive/Data/Combined_Images1.zip"
combined2_zip_path = "/content/drive/MyDrive/Data/Combined_Images2.zip"
combined3_zip_path = "/content/drive/MyDrive/Data/Combined_Images3.zip"

with ZipFile(combined1_zip_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images1")
    print('Done unzipping Combined_Images1.zip')
with ZipFile(combined2_zip_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images2")
    print('Done unzipping Combined_Images2.zip')
with ZipFile(combined3_zip_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images3")
    print('Done unzipping Combined_Images3.zip')        

Done unzipping Combined_Images1.zip
Done unzipping Combined_Images2.zip
Done unzipping Combined_Images3.zip


In [4]:
#1. Load Dataset
def load_dataset(folder_name, n_range):
    images = []
    images_path = glob.glob('../content/Combined_Images{}/{}/*.jpg'.format(n_range, str(folder_name)))
    for img_path in images_path:
        img = cv2.imread(img_path)
        img = cv2.resize(img, dsize=(50, 50))
        images.append(img)   
    return images  

In [5]:
def flipping(images):
    #1. --------- Flipping Augmentation ---------
    flipping_augmentation = iaa.Fliplr(1)
    flipped_images = flipping_augmentation(images=images)
    return flipped_images

def rotate_left(images):
    #2. --------- Rotating Left Augmentation ---------
    rotate_left_augmentation = iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                          rotate=(-30, -5),
                                          scale=(0.5, 1.5))
    rotate_left_images = rotate_left_augmentation(images=images)
    return rotate_left_images

def rotate_right(images):
    #3. --------- Rotating Right Augmentation ---------
    rotate_right_augmentation = iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                           rotate=(5, 30),
                                           scale=(0.5, 1.5))
    rotate_right_images = rotate_right_augmentation(images=images)
    return rotate_right_images

def multiply_images(images):
    #4. --------- Multiply --------- to let the image lightner or darker
    multiply_augmentation = iaa.Multiply((0.5, 1.4)) #This is a lightner range
    multiply_images = multiply_augmentation(images=images)
    return multiply_images

def contrast(images):
    #5. --------- Contrast ---------
    contrast_augmentation = iaa.LinearContrast((1.6, 2.4))
    contrast_images = contrast_augmentation(images=images)
    return contrast_images

def blur(images):
    #6. --------- Blur ---------
    blur_augmentation = iaa.GaussianBlur((0.5, 1.5))
    blur_images = blur_augmentation(images=images)
    return blur_images

In [6]:
dirpath = ["/content/Combined_Faces_Augmented1/", 
            "/content/Combined_Faces_Augmented2/", 
            "/content/Combined_Faces_Augmented3/"]        

In [7]:
if not os.path.exists("Combined_Faces_Augmented1"):
    os.mkdir("Combined_Faces_Augmented1")
if not os.path.exists("Combined_Faces_Augmented2"):
    os.mkdir("Combined_Faces_Augmented2")
if not os.path.exists("Combined_Faces_Augmented3"):
    os.mkdir("Combined_Faces_Augmented3")

In [8]:
#REPEAT THIS PART TO CREATE ALL THE 8 SUBFOLDERS
for idx, folder in enumerate(dirpath):
    for num in range(7): #range(1)=range(7)
        os.mkdir('Combined_Faces_Augmented{}/'.format(idx+1) + str(num))
        path = folder + str(num)
        images = load_dataset(str(num), idx+1)
        tot_imgs = images
        print('---- START IMAGES PROCESSING CLASS {} ----'.format(num))
        for n in range(len(images)):
            if n % 500 == 0:
                print('{} images processed of {}'.format(n, len(images)))
            flp = flipping([images[n]])
            rot_left = rotate_left([images[n]])
            rot_right = rotate_right([images[n]])
            mult = multiply_images([images[n]])
            contr = contrast([images[n]])
            blr = blur([images[n]])
            aug_img = [flp, rot_left, rot_right, mult, contr, blr]
            tot_imgs = tot_imgs + aug_img
        print(len(tot_imgs))    
        print('Image processing for class {} completed\n'.format(num))    
        print('---- START AUGMENTED IMAGES PROCESS CLASS {} ----'.format(num))                 
        for index, img in enumerate(tot_imgs):
            if isinstance(img, list):
                img = img.pop()
            cv2.imwrite(path + '/{}.jpg'.format(index), img)
            if index % 500 == 0:
              print('{} images created of {}'.format(index, len(tot_imgs)))
        print('Augmented images creation process for class {} completed\n'.format(num)) 

---- START IMAGES PROCESSING CLASS 0 ----
0 images processed of 5441
500 images processed of 5441
1000 images processed of 5441
1500 images processed of 5441
2000 images processed of 5441
2500 images processed of 5441
3000 images processed of 5441
3500 images processed of 5441
4000 images processed of 5441
4500 images processed of 5441
5000 images processed of 5441
38087
Image processing for class 0 completed

---- START AUGMENTED IMAGES PROCESS CLASS 0 ----
0 images created of 38087
500 images created of 38087
1000 images created of 38087
1500 images created of 38087
2000 images created of 38087
2500 images created of 38087
3000 images created of 38087
3500 images created of 38087
4000 images created of 38087
4500 images created of 38087
5000 images created of 38087
5500 images created of 38087
6000 images created of 38087
6500 images created of 38087
7000 images created of 38087
7500 images created of 38087
8000 images created of 38087
8500 images created of 38087
9000 images created

In [9]:
#Creo le 3 cartelle Combined_Images_Augmented.zip e la inserisco sul Google Drive
aug_path1 = "../content/Combined_Faces_Augmented1"
output_filename = '../content/drive/MyDrive/Data/Combined_Images_Augmented1'
shutil.make_archive(output_filename, 'zip', aug_path1)

aug_path2 = "../content/Combined_Faces_Augmented2"
output_filename = '../content/drive/MyDrive/Data/Combined_Images_Augmented2'
shutil.make_archive(output_filename, 'zip', aug_path2)

aug_path3 = "../content/Combined_Faces_Augmented3"
output_filename = '../content/drive/MyDrive/Data/Combined_Images_Augmented3'
shutil.make_archive(output_filename, 'zip', aug_path3)

'/content/drive/MyDrive/Data/Combined_Images_Augmented3.zip'